In [1]:
import kagglehub
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping

c:\Users\baki_\.conda\envs\movie_predict\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# === 1. VERİ YÜKLEME ===
path = kagglehub.dataset_download("parasharmanas/movie-recommendation-system")
movies = pd.read_csv(path+"/movies.csv")
ratings = pd.read_csv(path+"/ratings.csv")

In [3]:
# === 2. VERİ HAZIRLAMA ===
data = ratings.merge(movies, on="movieId")
data["genres"] = data["genres"].apply(lambda x: x.split('|'))

In [4]:
# Encode user/movie
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
data["user_enc"] = user_encoder.fit_transform(data["userId"])
data["movie_enc"] = movie_encoder.fit_transform(data["movieId"])

In [5]:
# Genre vectorizer
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(data["genres"])

In [6]:
# Eğitim girdileri
user_ids = data["user_enc"].values
movie_ids = data["movie_enc"].values
ratings_val = data["rating"].values

In [7]:
# === 3. MODEL MİMARİSİ ===
num_users = data["user_enc"].nunique()
num_movies = data["movie_enc"].nunique()
num_genres = genre_matrix.shape[1]

user_input = Input(shape=(1,), name="user_input")
movie_input = Input(shape=(1,), name="movie_input")
genre_input = Input(shape=(num_genres,), name="genre_input")

user_vec = Flatten()(Embedding(num_users, 64)(user_input))
movie_vec = Flatten()(Embedding(num_movies, 64)(movie_input))

x = Concatenate()([user_vec, movie_vec, genre_input])
x = Dense(128, activation="relu")(x)
x = Dropout(0.4)(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(32, activation="relu")(x)
output = Dense(1, name="rating_output")(x)

model = Model(inputs=[user_input, movie_input, genre_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0003), loss=Huber())

In [8]:
# === 4. ERKEN DURDURMA ===
early_stop = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

In [ ]:
# === 5. MODEL EĞİTİMİ ===
model.fit(
    [user_ids, movie_ids, genre_matrix],
    ratings_val,
    epochs=20,
    batch_size=256,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stop]
)

In [ ]:
# === 6. MODEL VE ENCODER'LARI KAYDET ===
model.save("movie_predict_model.h5")
print("✅ Model kaydedildi: movie_predict_model.h5")

with open("movie_encoder.pkl", "wb") as f:
    pickle.dump(movie_encoder, f)
with open("mlb.pkl", "wb") as f:
    pickle.dump(mlb, f)
print("✅ movie_encoder.pkl ve mlb.pkl kaydedildi.")
